<a href="https://colab.research.google.com/github/raffeekk/ML/blob/main/%D0%A2%D0%B5%D0%BC%D0%B0%205/assignment05_logit_rf_credit_scoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <center> Домашнее задание № 5. Решение
## <center> Логистическая регрессия и случайный лес в задаче кредитного скоринга

**Задание 1** В зале суда есть 5 присяжных, каждый из них по отдельности с вероятностью 70% может правильно определить, виновен подсудимый или нет. С какой вероятностью они все вместе вынесут правильный вердикт, если решение принимается большинством голосов?
- 70.00%
- 83.20%
- 83.70%
- 87.50%

Решение:
поскольку большинство голосов – 3, тогда у нас $m = 3,~N = 5,~p = 0.7$. Подставляем в формулу из статьи $$ \large \mu = \sum_{i=3}^{5}C_5^i0.7^i(1-0.7)^{5-i} $$
После подставления и проделывания всех операций получим ответ 83.70%

In [36]:
from scipy.stats import binom

# Задаем параметры
n = 5  # количество присяжных
p = 0.7  # вероятность правильного решения

# Определяем количество голосов, необходимых для большинства (здесь необходимо 3 голоса)
majority_needed = 3

# Вычисляем вероятность того, что большинство присяжных вынесет правильный вердикт
probability_correct_verdict = sum(binom.pmf(k, n, p) for k in range(majority_needed, n + 1))

# Преобразуем в процент
probability_correct_verdict_percentage = probability_correct_verdict * 100

print(f'Вероятность правильного вердикта присяжных: {probability_correct_verdict_percentage:.2f}%')


Вероятность правильного вердикта присяжных: 83.69%


In [38]:
%matplotlib inline
# отключим предупреждения Anaconda
import warnings

import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd

In [39]:
## Сделаем функцию, которая будет заменять NaN значения на медиану в каждом столбце таблицы
def delete_nan(table):
    for col in table.columns:
        table[col] = table[col].fillna(table[col].median())
    return table

In [41]:
## Считываем данные
credit_scoring_sample = "https://raw.githubusercontent.com/raffeekk/ML/refs/heads/main/%D0%A2%D0%B5%D0%BC%D0%B0%205/credit_scoring_sample.csv"
data = pd.read_csv(credit_scoring_sample, sep=";")
data.head()

,SeriousDlqin2yrs,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,NumberOfTimes90DaysLate,NumberOfTime60-89DaysPastDueNotWorse,MonthlyIncome,NumberOfDependents
0,0,64,0,0.249908,0,0,8158.0,0.0
1,0,58,0,3870.000000,0,0,NaN,0.0
2,0,41,0,0.456127,0,0,6666.0,0.0
3,0,43,0,0.000190,0,0,10500.0,2.0
4,1,49,0,0.271820,0,0,400.0,0.0


In [42]:
independent_columns_names = data.columns.values
independent_columns_names = [x for x in data if x != 'SeriousDlqin2yrs']
independent_columns_names

['age',
 'NumberOfTime30-59DaysPastDueNotWorse',
 'DebtRatio',
 'NumberOfTimes90DaysLate',
 'NumberOfTime60-89DaysPastDueNotWorse',
 'MonthlyIncome',
 'NumberOfDependents']

In [43]:
table = delete_nan(data)
X = table[independent_columns_names]
y = table["SeriousDlqin2yrs"]

# Бутстрэп

**Задание 2.** Сделайте интервальную оценку среднего возраста (age) для клиентов, которые просрочили выплату кредита, с 90% "уверенностью". (используйте пример из статьи. Поставьте np.random.seed(0), как это сделано в статье).

In [44]:
def get_bootstrap_samples(data, n_samples):
    # функция для генерации подвыборок с помощью бутстрэпа
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples
def stat_intervals(stat, alpha):
    # функция для интервальной оценки
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

# сохранение в отдельные numpy массивы данных по просрочке
churn = data[data['SeriousDlqin2yrs'] == 1]['age'].values

# ставим seed для воспроизводимости результатов
np.random.seed(0)

# генерируем выборки с помощью бутстрэра и сразу считаем по каждой из них среднее
churn_mean_scores = [np.mean(sample)
                       for sample in get_bootstrap_samples(churn, 1000)]

#  выводим интервальную оценку среднего
print("Mean interval",  stat_intervals(churn_mean_scores, 0.1))

Mean interval [45.71379414 46.12700479]


# Логистическая регрессия

In [45]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, StratifiedKFold
scaler = StandardScaler()
lr = LogisticRegression(random_state=5, class_weight="balanced", max_iter=200)

parameters = {"C": (0.0001, 0.001, 0.01, 0.1, 1, 10)}
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=5)

**Задание 3.**
Сделайте GridSearch с метрикой "roc-auc" по параметру C. Какое оптимальное значение параметра C получилось?



In [46]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, StratifiedKFold

# Определение модели
lr = LogisticRegression(solver='liblinear')

# Задаем диапазон значений для параметра C
parameters = {'C': [0.001, 0.01, 0.1, 1, 10, 100]}

# Создаем кросс-валидатор
skf = StratifiedKFold(n_splits=5)

# Выполняем GridSearch
grid_search = GridSearchCV(lr, parameters, n_jobs=-1, scoring='roc_auc', cv=skf)
grid_search.fit(X, y)

# Получаем оптимальное значение параметра C
optimal_C = grid_search.best_estimator_.C
print(f'Оптимальное значение параметра C: {optimal_C}')


Оптимальное значение параметра C: 100


**Задание 4.**
Можно ли считать лучшую модель устойчивой? (модель считаем устойчивой, если стандартное отклонение на валидации меньше 0.5%)

In [47]:
grid_search.cv_results_['std_test_score'][1]

0.02244049022970726

In [48]:
grid_search.best_score_

0.7338128017000611

### Определение влияния признаков

**Задание 5.** Определите самый важный признак. Важность признака определяется абсолютным значением его коэффициента. Также нужно масштабировать все признаки, чтобы можно было корректно сравнивать коэффициенты при них.

In [49]:
from sklearn.preprocessing import StandardScaler
lr = LogisticRegression(C=0.001,random_state=5, class_weight= 'balanced')
scal = StandardScaler()
lr.fit(scal.fit_transform(X), y)

pd.DataFrame({'feat': independent_columns_names,
              'coef': lr.coef_.flatten().tolist()}).sort_values(by='coef', ascending=False)

,feat,coef
1,NumberOfTime30-59DaysPastDueNotWorse,0.723421
3,NumberOfTimes90DaysLate,0.516656
4,NumberOfTime60-89DaysPastDueNotWorse,0.195486
6,NumberOfDependents,0.101722
2,DebtRatio,-0.024068
5,MonthlyIncome,-0.163015
0,age,-0.417115


Самый важный признак – NumberOfTime30-59DaysPastDueNotWorse.

**Задание 6.** Посчитайте долю влияния `DebtRatio` на предсказание (Реализуйте функцию [softmax](https://en.wikipedia.org/wiki/Softmax_function)).

In [50]:
print((np.exp(lr.coef_[0]) / np.sum(np.exp(lr.coef_[0])))[2])

0.11423641577786516


**Задание 7.**
Давайте посмотрим, как можно интерпретировать влияние наших признаков. Для этого заново оценим логистическую регрессию в абсолютных величинах. После этого посчитайте, во сколько раз увеличатся шансы, что клиент не выплатит кредит, если увеличить возраст на 20 лет при всех остальных равных значениях признаков (теоретический расчет можно посмотреть [здесь](https://www.unm.edu/~schrader/biostat/bio2/Spr06/lec11.pdf)).

In [51]:
lr = LogisticRegression(C=0.001,random_state=5, class_weight= 'balanced')
lr.fit(X, y)

pd.DataFrame({'feat': independent_columns_names,
              'coef': lr.coef_.flatten().tolist()}).sort_values(by='coef', ascending=False)

,feat,coef
1,NumberOfTime30-59DaysPastDueNotWorse,0.451547
3,NumberOfTimes90DaysLate,0.397109
4,NumberOfTime60-89DaysPastDueNotWorse,0.234869
6,NumberOfDependents,0.159904
5,MonthlyIncome,-0.000011
2,DebtRatio,-0.000015
0,age,-0.013064


In [52]:
np.exp(lr.coef_[0][0]*20)

0.7700690893655148

$\exp^{\beta\delta}$ – во столько раз больше шансы, что клиент не выплатит кредит. Где $\delta$ – на сколько делаем прирост. Например, если увеличить возраст на 20 лет, то шансы, что человек не выплатит кредит, увеличатся в 0.69.

# Случайный лес

In [53]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=42,
                            class_weight='balanced')

parameters = {'max_features': [1, 2, 4], 'min_samples_leaf': [3, 5, 7, 9], 'max_depth': [5,10,15]}

**Задание 8.** На сколько доля верных ответов лучшей модели случайного леса выше аналогичной метрики для логистической регрессии на валидации?

In [56]:
from sklearn.model_selection import GridSearchCV
rf_grid_search = GridSearchCV(rf, parameters, n_jobs=-1, scoring ='roc_auc', cv=skf,
                             verbose=True)
rf_grid_search = rf_grid_search.fit(X, y)
print(rf_grid_search.best_score_ - grid_search.best_score_)

Fitting 5 folds for each of 36 candidates, totalling 180 fits


KeyboardInterrupt: 

**Задание 9.** Определите, какой признак имеет самое слабое влияние.

In [ ]:
independent_columns_names[np.argmin(rf_grid_search.best_estimator_.feature_importances_)]

Весь рейтинг важности признаков

In [ ]:
pd.DataFrame({'feat': independent_columns_names,
              'coef': rf_grid_search.best_estimator_.feature_importances_}).sort_values(by='coef', ascending=False)

,feat,coef
1,NumberOfTime30-59DaysPastDueNotWorse,0.300290
3,NumberOfTimes90DaysLate,0.278749
4,NumberOfTime60-89DaysPastDueNotWorse,0.156534
0,age,0.115860
2,DebtRatio,0.076082
5,MonthlyIncome,0.057994
6,NumberOfDependents,0.014491


** Задание 10.** Какое наиболее существенное примущество логистической регрессии перед случайным лесом для нашей бизнес-задачи?

- меньше тратится времени для тренировки модели;
- меньше параметров для перебора;
- интепретируемость признаков;
- линейные свойства алгоритма.

В итоге мы получили, что алгоритм случайно леса лучше сработал для нашей задачи скоринга. Доля верных ответов случайного леса почти на 4% выше. Причинами такого результата стали – небольшое количество признаков и свойства случайного леса как композиции.

Но преимущество логистической регрессии в том, что мы можем проинтерпретировать влияние коэффициентов на результат.

# Бэггинг

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import cross_val_score, RandomizedSearchCV

parameters = {'max_features': [2, 3, 4], 'max_samples': [0.5, 0.7, 0.9], "base_estimator__C": [0.0001, 0.001, 0.01, 1, 10, 100]}

**Задание 11.** Следующая задача обучить – бэггинг классификатор. В качестве базовых классификаторов возьмите 100 логистических регрессий и на этот раз используйте не GridSearchCV, а RandomizedSearchCV. Так как перебирать все 54 варианта комбинаций долго, то поставьте максимальное число итераций 20 для RandomizedSearchCV. Также не забудьте передать параметр валидации cv и random_state=1. Какая лучшая доля верных ответов получилась?

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score

# Создаем объект BaggingClassifier с LogisticRegression в качестве базового оценивателя
bg = BaggingClassifier(estimator=LogisticRegression(class_weight='balanced', max_iter=500),
                       n_estimators=100, n_jobs=-1, random_state=42)

# Задаем параметры для RandomizedSearchCV
parameters = {
    'estimator__C': [0.0001, 0.001, 0.01, 0.1, 1, 10]  # Используем 'estimator__C'
}

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=5)

# Инициализируем RandomizedSearchCV
r_grid_search = RandomizedSearchCV(bg, parameters, n_jobs=-1,
                                   scoring='roc_auc', cv=skf, n_iter=20,
                                   random_state=1, verbose=True)

# Обучаем модель
r_grid_search = r_grid_search.fit(X, y)

# Получаем предсказания на обучающей выборке или на отдельной тестовой выборке
y_pred = r_grid_search.predict(X)

# Вычисляем долю верных ответов
accuracy = accuracy_score(y, y_pred)

print(f'Лучшая доля верных ответов: {accuracy:.4f}')


Fitting 5 folds for each of 6 candidates, totalling 30 fits
Лучшая доля верных ответов: 0.8168


In [ ]:
r_grid_search.best_score_

0.806653788888482

**Задача 12.** Дайте интерпретацию лучших параметров для бэггинга. Почему именно такие значения оказались лучшими?

- для бэггинга важно использовать как можно меньше признаков
- бэггинг лучше работает на небольших выборках
- меньше корреляция между одиночными моделями
- чем больше признаков, тем меньше теряется информации

Преимущество случайного леса в том, что деревья в композиции слабо коррелируют между собой и это дает максимальную пользу. Аналогично и для бэггинга с логистической регрессией, чем слабее коррелируют между собой одиночные модели, тем выше точность. Поскольку в логистической регрессии практически нет случайности, то мы должны менять наборы признаков и объектов для достижении максимальной декоррелированости наших моделей.